<a href="https://colab.research.google.com/github/charlesdgburns/NM_TinyRNN/blob/main/notebooks/understanding_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Let's use this as a learning notebook about different RNN architectures.
We want to start from scratch and get an idea of how different architectures work before fitting to some data.


We the fit these to sequential behavioural decision making later.


In [1]:
## setup on google colab:

!git clone https://github.com/charlesdgburns/NM_TinyRNN.git

Cloning into 'NM_TinyRNN'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 191 (delta 74), reused 112 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (191/191), 788.23 KiB | 7.30 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
# setup

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path


# Recurrent Neural Network (RNN)

The simplest of all recurrent architectures, which we think of as updating a hidden state $h_t$ using sensory inputs $x_{t-1}$ and recurrent inputs from the last hidden state $h_{t-1}$. Formally, this is described as a single discretised update:

$$ h_t = \tanh(W_{ih} x_{t-1} + b_{i} + W_{hh} h_{t-1} +b_{hh}) $$

where $W_{ih}$ are the weights from the input to the hidden state, $W_{hh}$ are the recurrent weights from the hidden state to itself, and we consider additional biases $b_{i}$ and $b_{hh}$.

In [ ]:
# let's do this later.

# Gated Recurrent Unit (GRU)

We are taking inspiration from the following blogpost:
https://medium.com/data-science/building-a-lstm-by-hand-on-pytorch-59c02a4ec091

**The information flow in a gated recurrent unit**

A gated recurrent unit has information flowing from the inputs $^{(1)}$ $x_{t-1}$  and the past hidden $h_{t-1}$, which is gated via 'reset' and 'update' gates $r_t$ and $z_t$ before giving the final output $h_t$. The update gate decides whether or not to overwrite a long-term memory with inputs, while the

The gated recurrent unit allows the recurrent unit to persist its state and ignore its inputs.



\begin{align}
r_t &= \sigma(W_{ir}x_{t-1}+W_{hr}h_{t-1}+b_{r})  &&\text{(reset gate)} \\
z_t &= \sigma(W_{iz}x_{t-1}+W_{hz}h_{t-1}+b_{z})  &&\text{(update gate)} \\
n_t &= \tanh(W_{in}x_{t-1}+b_{in}+r_{t}\cdot (W_{hn}h_{t-1}+b_{hn})) &&\text{(new state)}
\\
h_t &= (1-z_t)\cdot h_{t-1} +z_t \cdot n_t &&\text{(hidden state)}
\end{align}

*footnotes*

Note that GRU's are markovian - their current state ($h_t$) can be determined entirely from its previous state ($h_{t-1}$) and inputs ($x_{t-1}$).

In [ ]:
import torch
import torch.nn as nn
import math
import numpy as np


class ManualGRU(nn.Module):
  '''Manual GRU coded to return gate activations dictionary
  Returns: (hidden_sequence, h_past) if return_gate_activations=False,
           (hidden_sequence, gate_activations) if -||- = True'''
  def __init__(self,input_size,hidden_size):
    super().__init__() #init nn.Module
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.sigmoid = torch.nn.Sigmoid()
    self.tanh = torch.nn.Tanh()

    self.W_from_in = nn.Parameter(torch.Tensor(input_size, hidden_size*3))
    self.W_from_h = nn.Parameter(torch.Tensor(hidden_size, hidden_size*3))
    self.bias = nn.Parameter(torch.Tensor(hidden_size*6))
    self.init_weights()

  def init_weights(self):
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
        weight.data.uniform_(-stdv, stdv)

  def forward(self, inputs, init_states = None,
              return_gate_activations=False):
    ''' inputs are a tensor of shape (batch_size, sequence_size, input_size)
        outputs are tensor of shape (batch_size, sequence_size, hidden_size)'''

    batch_size, sequence_size, _ = inputs.shape
    hidden_sequence = []
    if return_gate_activations:
      gate_activations = {'reset':[],'update':[]}
    if init_states is None:
      h_past = torch.zeros(batch_size, self.hidden_size).to(inputs.device) #(n_hidden,batch_size)
    else:
      h_past = init_states

    for t in range(sequence_size):
      x_past = inputs[:,t,:] #(n_batch,input_size)
      #for computational efficiency we do two matrix multiplications and then do indexing further down:
      from_input = x_past@self.W_from_in + self.bias[:3*self.hidden_size]
      from_input = from_input.view(batch_size,3, self.hidden_size) #(n_batch,3,n_hidden)
      from_hidden = h_past@self.W_from_h + self.bias[3*self.hidden_size:]
      from_hidden = from_hidden.view(batch_size, 3, self.hidden_size) #(n_batch,3, n_hidden)
      r_t = self.sigmoid(from_input[:,0]+from_hidden[:,0]) #(n_batch,n_hidden), ranging from 0 to 1
      z_t = self.sigmoid(from_input[:,1]+from_hidden[:,1]) #(n_batch,n_hidden), ranging from 0 to 1; must have n_hidden because it is multiplied with hidden_state later.
      if return_gate_activations:
        gate_activations['reset'].append(r_t.unsqueeze(0))
        gate_activations['update'].append(z_t.unsqueeze(0))
      n_t = self.tanh(from_input[:,2]+r_t*(from_hidden[:,2])).view(batch_size, self.hidden_size) #(n_batch,n_hidden)
      h_past = (1-z_t)*n_t + z_t*h_past #(n_batch,hidden_size) #NOTE h_past is tehnically h_t now, but in the next for-loop it will be h_past. ;)
      hidden_sequence.append(h_past.unsqueeze(0)) #appending (1,n_batch,n_hidden) to a big list.

    hidden_sequence = torch.cat(hidden_sequence, dim=0) #(n_sequence, n_batch, n_hidden) gather all inputs along the first dimenstion
    hidden_sequence = hidden_sequence.transpose(0, 1).contiguous() #reshape to batch first (n_batch,n_seq,n_hidden)

    if return_gate_activations:
        for gate_label, activations in gate_activations.items():
            gate_activations[gate_label] = torch.cat(activations, dim=0).transpose(0,1).contiguous() #(n_batch,n_seq,n_hidden)
        return hidden_sequence, gate_activations
    else:
      return hidden_sequence, h_past #this is standard in Pytorch, to output sequence of hidden states alongside most recent hidden state.

test_inputs = torch.ones((8,150,3))
test = ManualGRU(3, 2)
hidden_sequence, h_past = test(test_inputs)
hidden_sequence, gate_activations = test(test_inputs,return_gate_activations=True)
gate_activations['reset'].shape

torch.Size([8, 150, 2])

# Long-Short-Term-Memory

Here we implement a standard LSTM layer by layer.

- Following https://medium.com/data-science/building-a-lstm-by-hand-on-pytorch-59c02a4ec091

### Maths:

\begin{align}
 f_t &=\sigma(W_{if} x_t+W_{hf} h_{t-1}+b_f) &&\text{(forget gate)}
 \\
 i_t &= \sigma(W_{ii} x_t+W_{ih} h_{t-1}+b_i)  &&\text{(input gate)}
 \\
 o_t &=  \sigma(W_{io}  x_t+W_{oh} h_{t-1}+b_o) &&\text{(output gate)}
 \\
 \tilde{C}_t & = \tanh(W_{ic}x_{t-1}+W_{hc}h_{t-1}+b_c) &&\text{(partial candidate)}
 \\
 C_t&=f_t\odot C_{t-1} + \tilde{C}_t &&\text{(candidate / LTM)}\\
 h_t & = o_t \odot \tanh(C_t) &&\text{(hidden / STM)}
\end{align}



## Classic blog post explanation goes like:
### 1. Forge gate

The forget gate is where input, $x_t$, is combined with long term memory (termed the 'candidate'), $C_{t-1}$, and short term memory (the hidden state), $h_{t-1}$ . That is:

$$ f_t =\sigma(U_f x_t+V_f h_{t-1}+b_f),\qquad C'_t=f_t\cdot C_{t-1}$$

where $\sigma\mapsto (0,1)$ is a sigmoidal activation function. The term $C'_t$ represents how much of the long term memory is retrieved.

### 2. Input gate

Where input informaion and hidden states are combined along with partial candidate $C'_t$ to form a new candidate:

$$ i_t = \sigma(U_ix_t+V_ih_{t-1}+b_i) \\
C_t^+=\tanh(U_c x_t+V_c h_{t-1} +b_c) \\
C_t= C'_t+i_t\cdot C_t^+$$

Note here that $\tanh\mapsto (-1,1)$ affects how much information will be introduced to the memory.

### 3. Output gate and hidden state of the cell

Lastly we combine the l short term memory ($h_{t-1}$) and inputs ($x_t$) at the output gate, which is finally combined with long term memory (C_t) to get out final output ($h_t$).

$$o_t=\sigma(U_o  x_t+V_o h_{t-1}+b_o) \\ h_t = o_t\cdot \tanh(C_t) $$

In [ ]:



class ManualLSTM(nn.Module):
    def __init__(self, input_sz, hidden_sz, device='cpu'):
        super().__init__()
        self.device=device
        self.input_sz = input_sz
        self.hidden_size = hidden_sz
        self.W = nn.Parameter(torch.Tensor(input_sz, hidden_sz * 4))
        self.U = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz * 4))
        self.bias = nn.Parameter(torch.Tensor(hidden_sz * 4))
        self.init_weights()


    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv).to(self.device)

    def forward(self, x, init_states=None,
                return_gate_activations = False):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size() #let's get our shapes right
        HS = self.hidden_size
        if init_states is None:
            h_t, c_t = (torch.zeros(bs, self.hidden_size).to(x.device),
                        torch.zeros(bs, self.hidden_size).to(x.device))
        else:
            h_t, c_t = init_states

        hidden_sequence = []#initialise lists for unit activations
        if return_gate_activations:
            gate_activations = {'input':[],'forget':[],'output':[], 'candidate':[]}
        for t in range(seq_sz):
            x_t = x[:, t, :]
            # batch the computations into a single matrix multiplication
            gates = x_t @ self.W + h_t @ self.U + self.bias
            i_t, f_t, g_t, o_t = (
                torch.sigmoid(gates[:, :HS]), # input
                torch.sigmoid(gates[:, HS:HS*2]), # forget
                torch.tanh(gates[:, HS*2:HS*3]), # this integrates inputs and past hidden (so over short-term memory)
                torch.sigmoid(gates[:, HS*3:]), # output
            )
            c_t = f_t * c_t + i_t * g_t #here we integrate inputs/STM with long-term memory// never update candidate if forget = 1 and input = 0.
            h_t = o_t * torch.tanh(c_t)
            hidden_sequence.append(h_t.unsqueeze(0))
            if return_gate_activations:
                for gate_label, activation in {'input':i_t,'forget':f_t,'output':o_t,'candidate':c_t}.items():
                    gate_activations[gate_label].append(activation.unsqueeze(0))
        hidden_sequence = torch.cat(hidden_sequence, dim=0) #(sequence, batch, n_hidden)
        hidden_sequence = hidden_sequence.transpose(0, 1).contiguous() #(batch, sequence, feature)
        if return_gate_activations:
            for gate_label, activations in gate_activations.items():
                gate_activations[gate_label] = torch.cat(activations, dim=0).transpose(0,1).contiguous() #(n_batch,n_seq,n_hidden)
            return hidden_sequence, gate_activations
        else:
            return hidden_sequence, (h_t, c_t)

test_inputs = torch.ones((8,10,3))
testLSTM = ManualLSTM(3,2)
hidden_seq, hidden_tuple = testLSTM(test_inputs)
hidden_seq, hidden_tuple = testLSTM(test_inputs, init_states = hidden_tuple)
hidden_seq, gate_activations = testLSTM(test_inputs,
                                        return_gate_activations = True)


# NM-RNN

[Costacura et al. (2024)](https://openreview.net/pdf?id=HbIBqn3grD) introduced NM-RNNs to bridge a gap between more standard RNNs today and biophysical models.

Here we rewrite the equations (1) to (4) in Costacura et al. (2024) more similarly to the standard RNN notation above.

We have inputs $x_{t-1}$ and our past hidden state $h_{t-1}$ which we want to integrate to get a new hidden state $h_t$. However, we want to selectively change (by a gain) the weights of our recurrent network depending on a neuromodulation signal $s(z(t))$.

We therefore have a coupled network system, starting from a subnetwork state $z(t)$. Consider the following discretisation (see footnote below for comparison to costacurta et al's paper):

$$ z_t = \phi ( W_{zz}(z_{t-1})+W_{iz} x_{t-1} ) \qquad (1; \text{nm subnetwork}) $$

$$ h_{t-1} =\phi( W_{hh}(z_t)h_{t-1}+W_{ih} x_{t_i} )\qquad (2; \text{recurrent output})$$

$$s(z(t)) = \sigma(W_{zk} z_t+ b_k) \qquad (3; \text{modulation gating}) $$
$$W_x(z_t)=\sum_{k=1}^K s_k(z_t)\mathcal{l}_k r_k^T \qquad (4; \text{recurrent weight modulation})$$

Note that instead of a low-rank recurrent weight component, we want a tiny RNN, so we could modulating the weights associated with a given unit. Now the 'dynamic modes' are not the low ranks of a large network, but the activity of single units.


Importantly, in 1D we have at most a 1-rank decomposition as the recurrent weights, $W_{hh} \in \mathbb{R}$, are a scalar.

In 2D, if our two units are $A$ and $B$, we now have $W_{hh} \in \mathbb{R}^{2\times 2}$, a 2x2 matrix.
This matrix can now be decomposed into a 2-rank decomposition, each component representing an orthogonal direction in the recurrent dynamics of the 2D hidden state.

$$ W_{hh} =
\begin{bmatrix}
W_{AA} & W_{AB} \\
W_{BA} & W_{BB} \\
\end{bmatrix} = \mathbb{l}_1 r_1^{T} + \mathbb{l}_2 r_2^{T} + \mathcal{e}
$$

in practice we use an SVD decomposition of $W_{hh}$.

However, since we have a tiny RNN, we can imagine other decompositions of $W_{hh}$, such as column-wise or row-wise.
$$ W_{hh} =  
\begin{bmatrix}
W_{AA} & W_{AB} \\
W_{BA} & W_{BB} \\
\end{bmatrix} =
\begin{bmatrix}
W_{AA} & 0\\
W_{BA} & 0\\
\end{bmatrix} +
\begin{bmatrix}
0 & W_{AB}\\
0 & W_{BB} \\
\end{bmatrix}
=
\begin{bmatrix}
W_{AA} & W_{AB}\\
0 & 0\\
\end{bmatrix} +
\begin{bmatrix}
0 & 0\\
W_{BA} & W_{BB} \\
\end{bmatrix} $$

Now we can interpret scaling th column-wise decomposition as pre-synaptic modulation for each unit (first matrix controls recurrent inputs pre-synaptic to A, second matrix controls recurrent inputs pre-synaptic to B).
Similarly the row-wise decomposition allows us to modulate the post-synaptic recurrent activity of each unit.

You could even go as far as have separate neuromodulation for each synapse in the recurrent network.

So we have two slightly subtle different parameters here - one is the number of subnetwork units, and for each subnetwork unit is the dimensionality of the neuromodulation onto the hidden network. For example, with two hidden units, $h_t \in \mathbb{R}^2$, we could have a single neuromodulator $z_t \in \mathbb{R}$ which affects all the recurrent weights the same, $W_x(z(t))\in \mathbb{R}$


*FOOT:*
To keep our notation consistent across architectures, we make the following changes to costacurta et al.'s notation:
- $z_t$ is still subnetwork state, but instead of applying relu before weight multiplications, we do so after (this is equivalent in discretised time)
- $h_t$ refers our hidden state (the activity which is later decoded for an action) instead of $x_t$ in costacurta et al.
- $x_t$ refers to sensory input instead of $u_t$ in costacurta et al.
- $W_{xy}$ refers to weight(s) from $x$ to $y$, replacing the A,B,C naming convention in costacurta et al..



$$
W = U \Sigma V^\top = \sum_{i=1}^{\mathrm{rank}(W)} \sigma_i \, u_i v_i^\top,
\qquad
W = l_1 r_1^\top + l_2 r_2^\top \qquad \text{(standard SVD)}
$$

Now we add neuromodulation by $s_t$, which is a number between 0 and 1 for each index $s_i$, so we have

$$ W_{rec} = s_1 \cdot (l_1 r_1^\top) + s_2\cdot(l_2 r_2^\top) $$

In [ ]:
# Different decomopositions and example modulations

import torch
test_vector1 = torch.tensor([1,0]) #This is one extreme state of s_t for a 2d matrix.

test_matrix = torch.tensor([[1,2],[2,1]])
# Row-wise (post-synaptic) scaling: multiply each row of the matrix by the corresponding element in the vector
row_scaled = test_matrix * test_vector1.unsqueeze(1)
print("Row-wise scaled matrix:\n", row_scaled)
# Column-wise (pre-synaptic) scaling: multiply each column of the matrix by the corresponding element in the vector
col_scaled = test_matrix * test_vector1
print("Column-wise scaled matrix:\n", col_scaled)

U, S, Vh = torch.linalg.svd(test_matrix.float())
component_1 = torch.outer(S[0]*U[:, 0], Vh[0, :])
component_2 = torch.outer(S[1]*U[:, 1], Vh[1, :])

#fig, ax = plt.subplots(1,3)
#ax[0].imshow(test_matrix)
#ax[1].imshow(component_1)
#ax[2].imshow(component_2)
print(test_matrix)
component_1+component_2

Row-wise scaled matrix:
 tensor([[1, 2],
        [0, 0]])
Column-wise scaled matrix:
 tensor([[1, 0],
        [2, 0]])
tensor([[1, 2],
        [2, 1]])


tensor([[1.0000, 2.0000],
        [2.0000, 1.0000]])

In [ ]:
import torch
import torch.nn as nn
import math

class ManualNMRNN(nn.Module):
    def __init__(self, input_size, nm_size, nm_dim, hidden_size, nm_mode='column'):
        super().__init__()
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

       # assert nm_dim <= nm_size, "there must be at least as many subnetwork units as nm_dim size"
        self.nm_size = nm_size
        self.nm_dim = nm_dim
        self.hidden_size = hidden_size
        self.nm_mode = nm_mode  # 'low_rank', 'column', 'row'

        # Parameters
        self.W_ih = nn.Parameter(torch.Tensor(input_size, hidden_size))      # (I,H)
        self.W_hh = nn.Parameter(torch.Tensor(hidden_size, hidden_size))     # (H,H)
        self.W_iz = nn.Parameter(torch.Tensor(input_size, nm_size))          # (I,Zm)
        self.W_zz = nn.Parameter(torch.Tensor(nm_size, nm_size))             # (Zm,Zm)
        self.W_zk = nn.Parameter(torch.Tensor(nm_size, nm_dim))              # (Zm,K)
        self.bias_k = nn.Parameter(torch.Tensor(nm_dim))                     # (K,)

        self.init_weights()

    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size if self.hidden_size > 0 else 1.0)
        for p in self.parameters():
            p.data.uniform_(-stdv, stdv)

    def forward(self, inputs, init_states=None):
        """
        inputs:  (B, T, I)
        returns: hidden_sequence (B, T, H), (h_T, z_T)
        """
        device = inputs.device
        B, T, _ = inputs.size()
        H = self.hidden_size
        Zm = self.nm_size
        K = self.nm_dim

        # Initial states
        if init_states is None:
            h_past = torch.zeros(B, H, device=device)
            z_past = torch.zeros(B, Zm, device=device)   # <-- important: size nm_size
        else:
            h_past, z_past = init_states

        hidden_sequence = []

        # --- Precompute anything that doesn't depend on time ---
        # nothing mandatory, except SVD only needed if low_rank is used.
        # We'll compute low-rank components on-demand below.

        for t in range(T):
            x_t = inputs[:, t, :]                                   # (B,I)

            # Subnetwork dynamics -> modulation signal s_t
            # z_t: (B,Zm), s_t: (B,K)
            z_t = self.tanh(z_past @ self.W_zz + x_t @ self.W_iz)   # (B, Zm)
            s_t = self.sigmoid(z_t @ self.W_zk + self.bias_k)       # (B, K)

            # Build batched recurrent weight W_rec: (B,H,H)
            if self.nm_mode == 'low_rank':
                W_rec = self._make_Wrec_low_rank(s_t)               # (B,H,H)
            elif self.nm_mode == 'column':
                W_rec = self._make_Wrec_column(s_t)                 # (B,H,H)
            elif self.nm_mode == 'row':
                W_rec = self._make_Wrec_row(s_t)                    # (B,H,H)
            else:
                raise ValueError(f"Unknown nm_mode: {self.nm_mode}")
            # Recurrent step: h_t = tanh( h_{t-1} @ W_rec + x_t @ W_ih )
            # Use einsum for batched (B,H) × (B,H,H) -> (B,H)
            h_recur = torch.einsum('bi,bij->bj', h_past, W_rec)     # (B,H)
            h_t = self.tanh(h_recur + x_t @ self.W_ih )             # (B,H)
            hidden_sequence.append(h_t.unsqueeze(1))                # (B,1,H)
            # Update states
            h_past = h_t
            z_past = z_t

        hidden_sequence = torch.cat(hidden_sequence, dim=1)         # (B,T,H)
        return hidden_sequence, (h_past, z_past)

    def _make_Wrec_low_rank(self, s_t):
        """
        Low-rank modulation
        s_t: (B, K)  where K = nm_dim
        Build W_rec[b] = sum_{k=1..K} s_t[b,k] * sigma_k * u_k v_k^T
        """
        H = self.hidden_size
        K = self.nm_dim
        assert K <= H, "nm_dim (number of components) cannot exceed hidden_size"

        # SVD once per forward pass (on CPU/GPU depending on parameter device)
        U, S, Vh = torch.linalg.svd(self.W_hh, full_matrices=False)  # U:(H,H), S:(H,), Vh:(H,H)
        # take top-K components
        U_k = U[:, :K]                 # (H,K)
        S_k = S[:K]                    # (K,)
        Vh_k = Vh[:K, :]               # (K,H)

        # Prebuild the K rank-1 components C[k] = (S_k[k] * U_k[:,k]) ⊗ Vh_k[k,:]
        # L = (H,K), R = (K,H) -> C = (K,H,H)
        L = U_k * S_k.unsqueeze(0)     # (H,K) broadcast S_k across rows
        C = torch.einsum('ik,kj->kij', L, Vh_k)  # (K,H,H)

        # Combine per-batch with weights s_t: (B,K) × (K,H,H) -> (B,H,H)
        W_rec = torch.einsum('bk,kij->bij', s_t, C)
        return W_rec

    def _make_Wrec_column(self, s_t):
        """
        Column-wise modulation
        - If s_t has shape (B,1): global scalar per batch: W_rec[b] = s_t[b]*W_hh
        - If s_t has shape (B,H): per-column scaling: W_rec[b,:,j] = s_t[b,j] * W_hh[:,j]
        """
        B = s_t.shape[0]
        H = self.hidden_size
        W = self.W_hh.unsqueeze(0).expand(B, H, H)       # (B,H,H)
        if s_t.shape[1] == 1:
            return W * s_t.view(B, 1, 1)                 # scalar per batch
        elif s_t.shape[1] == H:
            return W * s_t.view(B, 1, H)                 # broadcast across rows -> scale columns
        else:
            raise ValueError(f"column mode expects nm_dim==1 or nm_dim==hidden_size ({H}), got {s_t.shape[1]}")

    def _make_Wrec_row(self, s_t):
        """
        Row-wise modulation
        - If s_t has shape (B,1): global scalar per batch: W_rec[b] = s_t[b]*W_hh
        - If s_t has shape (B,H): per-row scaling: W_rec[b,i,:] = s_t[b,i] * W_hh[i,:]
        """
        B = s_t.shape[0]
        H = self.hidden_size
        W = self.W_hh.unsqueeze(0).expand(B, H, H)       # (B,H,H)
        if s_t.shape[1] == 1:
            return W * s_t.view(B, 1, 1)                 # scalar per batch
        elif s_t.shape[1] == H:
            return W * s_t.view(B, H, 1)                 # broadcast across cols -> scale rows
        else:
            raise ValueError(f"row mode expects nm_dim==1 or nm_dim==hidden_size ({H}), got {s_t.shape[1]}")


test_inputs = torch.ones((8,10,3))
testNMRNN = ManualNMRNN(input_size = 3,
                       nm_size = 1,
                       nm_dim = 1,
                       hidden_size = 1,
                       nm_mode = 'low_rank')
hidden_seq, hidden_tuple = testNMRNN(test_inputs)
hidden_seq, hidden_tuple = testNMRNN(test_inputs, init_states = hidden_tuple)
#hidden_seq, gate_activations = testLSTM(test_inputs,
#                                        return_gate_activations = True)
hidden_seq.shape

torch.Size([8, 10, 1])

### Low-rank decomposition

The reason why an NM-RNN scales the low-rank components of the weight matrix, is that it allows it to selectively scale the different dimensions of the recurrent update.

Wait a second - what is a low-rank decomposition?
Let's take a look at one.

We have an example of a very simple 2-unit network with different recurrent dynamics.

In [ ]:
# Potentially useful examples of recurrent weights affecting dynamics

import numpy as np
import matplotlib.pyplot as plt

# ---------- Parameter sets (from your MATLAB) ----------
single = {
    "I_y": np.array([-10, -10]),
    "I_t1": np.array([0, 20]),
    "I_t2": np.array([20, 0]),
    "W": np.array([[-0.2, 0.8],
                   [ 0.8,-0.2]]),
    "r_init": np.array([50, 50]),
}

multi = {
    "I_y": np.array([-10, -5]),
    "I_t1": np.array([0, 20]),
    "I_t2": np.array([-20, 0]),
    "W": np.array([[ 1.1,-0.15],
                   [-0.15, 1.1]]),
    "r_init": np.array([50, 50]),
}

continuous = {
    "I_y": np.array([-10, -10]),
    "I_t1": np.array([0.5, 0.0]),
    "I_t2": np.array([0.5, 0.0]),
    "W": np.array([[ 0.8,-0.2],
                   [-0.2, 0.8]]),
    "r_init": np.array([50, 50]),
}

oscillator = {
    "I_y": np.array([ 8, 20]),
    "I_t1": np.array([0, 0]),
    "I_t2": np.array([0,10]),
    "W": np.array([[ 2.2,-1.3],
                   [ 1.2,-0.1]]),
    "r_init": np.array([50, 50]),
}

# --------- choose your configuration (match MATLAB: p = continuous) ----------
p = oscillator  # or: single, multi, oscillator

# ---------- Simulation settings (match MATLAB) ----------
dt = 1              # ms
t_max = 3000        # ms
duration = 250      # ms (length of each stimulation)
t1 = 1000           # ms
t2 = 2000           # ms
dt_tau = 0.1        # Euler ratio for 1 ms step and tau = 10 ms

# ---------- Build inputs over time (shape: T+1 by 2) ----------
T = int(t_max / dt)
times = np.arange(0, T + 1)  # 0..3000 inclusive -> 3001 samples
inputs = np.zeros((T + 1, 2))

idx1 = int(t1 / dt)
idx2 = int(t2 / dt)
dur = int(duration / dt)

inputs[idx1:idx1 + dur, :] = p["I_t1"]
inputs[idx2:idx2 + dur, :] = p["I_t2"]

# ---------- Fire-rate simulation (two units, linear-threshold with clipping) ----------
N = np.zeros((T + 1, 2))
N[0, :] = p["r_init"]

W = p["W"]
I_y = p["I_y"]

for k in range(T):  # update N[k+1] from N[k]
    # neuron 1
    N[k+1, 0] = N[k, 0] + dt_tau * (-N[k, 0] + W[0, :] @ N[k, :] + inputs[k, 0] - I_y[0])
    # neuron 2
    N[k+1, 1] = N[k, 1] + dt_tau * (-N[k, 1] + W[1, :] @ N[k, :] + inputs[k, 1] - I_y[1])

    # rectify to [0, 100] Hz
    N[k+1, :] = np.clip(N[k+1, :], 0.0, 100.0)

# ---------- Dynamics classification (same formulas as your MATLAB snippet) ----------
trace = -2 + W[0,0] + W[1,1]
det = (-1 + W[0,0]) * (-1 + W[1,1]) - W[0,1] * W[1,0]

stability = "unstable" if trace > 0 else "stable"
node_or_focus = "node" if trace**2 > 4 * det else "focus"

print(f"trace = {trace:.3f}")
print(f"det   = {det:.3f}")
print(f"Stability: {stability}")
print(f"Type: {node_or_focus}")

# ---------- Plot (3 panels similar to MATLAB layout) ----------
n1_colour = (0.5, 0.7, 1.0, 0.3)
n2_colour = (1.0, 0.4, 0.4, 0.3)

fig = plt.figure(figsize=(9, 5))
gs = fig.add_gridspec(3, 5)

# Firing rates over time: subplot(3,5,[6 7]) -> row=1, cols=0:2
ax_rates = fig.add_subplot(gs[1, 0:2])
ax_rates.plot(times, N[:, 0], color=n1_colour)
ax_rates.plot(times, N[:, 1], color=n2_colour)
ax_rates.set_xlim(0, T)
ax_rates.set_ylabel("Firing rate (Hz)")
ax_rates.set_title("Firing rates")

# Inputs over time: subplot(3,5,[11 12]) -> row=2, cols=0:2
ax_inputs = fig.add_subplot(gs[2, 0:2])
ax_inputs.plot(times, inputs[:, 0], color=n1_colour)
ax_inputs.plot(times, inputs[:, 1], color=n2_colour)
ax_inputs.set_xlim(0, T)
ax_inputs.set_ylabel("Injected currents")
ax_inputs.set_xlabel("Time (ms)")
ax_inputs.set_title("Inputs")

# Phase-plane: subplot(3,5,[4 5 9 10 14 15]) -> all rows, cols=3:5 (right block)
ax_phase = fig.add_subplot(gs[:, 3:5])
c = np.linspace(1, 10, N.shape[0])
ax_phase.scatter(N[:, 0], N[:, 1], s=5, c=c)
ax_phase.scatter(N[0, 0], N[0, 1], s=50, c='k', marker='+', label='start')
ax_phase.scatter(N[-1, 0], N[-1, 1], s=25, c='k', label='end')
ax_phase.set_xlim(0, 100)
ax_phase.set_ylim(0, 100)
ax_phase.set_xlabel("Neuron 1 Hz")
ax_phase.set_ylabel("Neuron 2 Hz")
ax_phase.set_aspect('equal', adjustable='box')
ax_phase.set_title("Phase-plane")
ax_phase.legend(loc="best", frameon=False)

plt.tight_layout()
plt.show()


trace = 0.100
det   = 0.240
Stability: unstable
Type: focus


: 

# Fitting to behaviour

We have behavioral data from a two-armed-bandit reversal task.

Mice are presented with two cue lights (A and B) and need to make a choice. Probabilities for rewards are 0.75 ('good poke') for one state and 0.25 on the other (not 'good poke'), a state which reverses randomly. However, occasionally only one light goes on (A or B) and the mouse must make their next poke there to continue the task, so this is considered 'forced_choice'.

Let's load some data:



In [ ]:
DATA_PATH = Path('./NM_TinyRNN/data/')
TASK = 'AB_behaviour'

subject_data = []
for subject_dir in (DATA_PATH/TASK).iterdir():
  if subject_dir.is_dir(): #skip the README file
    for session_dir in subject_dir.iterdir():
      if 'WS16' in session_dir.stem: #theres duplicate data with weird names/skip this
        continue
      if session_dir.is_dir(): #skip the .DS_session stuff
        subject_data.append(pd.read_csv(session_dir/'trials.htsv', sep = '\t'))
subject_df = pd.concat(subject_data) #concatenate across sessions here (that's ok)

subject_df

NameError: name 'Path' is not defined

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pathlib import Path

class AB_Dataset(Dataset):
    def __init__(self, subject_data_path, sequence_length):
        self.subject_data_path = Path(subject_data_path)
        self.sequence_length = sequence_length
        self.subject_df = self._load_and_concat_data()
        self.inputs, self.targets = self._create_sequences()

    def _load_and_concat_data(self):
        subject_data = []
        for session_dir in self.subject_data_path.iterdir():
            if 'WS16' in session_dir.stem:
                continue
            if session_dir.is_dir():
                subject_data.append(pd.read_csv(session_dir/'trials.htsv', sep = '\t'))
        return pd.concat(subject_data)

    def _create_sequences(self):
        # Convert boolean and categorical columns to numerical
        df_processed = self.subject_df.copy()
        df_processed['forced_choice'] = df_processed['forced_choice'].astype(int)
        df_processed['outcome'] = df_processed['outcome'].astype(int)
        df_processed['choice'] = df_processed['choice'].astype('category').cat.codes

        # Convert to tensor and handle potential remainder
        data_tensor = torch.tensor(df_processed[['forced_choice', 'outcome', 'choice']].values, dtype=torch.float32)
        num_rows = data_tensor.size(0)
        remainder = num_rows % self.sequence_length
        if remainder != 0:
            data_tensor = data_tensor[:-remainder] # Trim off the remainder

        # Reshape into sequences
        num_sequences = data_tensor.size(0) // self.sequence_length
        sequences = data_tensor.view(num_sequences, self.sequence_length, data_tensor.size(1))

        # Create inputs and targets
        # Inputs are 'forced_choice' and 'outcome' at time t
        inputs = sequences[:, :-1, :]
        # Targets are 'choice' at time t+1, one-hot encoded
        targets_codes = sequences[:, 1:, 2].long() # Get the categorical codes as long tensor
        targets = torch.nn.functional.one_hot(targets_codes, num_classes=2).float() # One-hot encode


        return inputs, targets


    def __len__(self):
        return self.inputs.size(0)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

# Example usage:
# Define the data path and sequence length
DATA_PATH = './NM_TinyRNN/data/AB_behaviour/WS16'
SEQUENCE_LENGTH = 150+1 # Define your desired sequence length

# Create the dataset and dataloader
dataset = AB_Dataset(DATA_PATH, SEQUENCE_LENGTH)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Example of iterating through the dataloader
# for batch_inputs, batch_targets in dataloader:
#      print("Inputs shape:", batch_inputs.shape)
#      print("Targets shape:", batch_targets.shape)
#      break # Just printing the first batch shapes for demonstration

In [ ]:
# Create a flexible way of training different RNNs to do the task prediction

class TinyRNN(nn.Module):
  def __init__(self, input_size:int=3, hidden_size:int=1, out_size:int=2,
               rnn_type = 'GRU',
               sparsity_lambda:float = 1e-2):
    super().__init__()
    self.I = input_size
    self.H = hidden_size
    self.O = out_size
    self.rnn_type = rnn_type
    if rnn_type == 'GRU':
      self.rnn = ManualGRU(self.I,self.H)
    elif rnn_type == 'LSTM':
      self.rnn = ManualLSTM(self.I,self.H)
    elif rnn_type == 'NMRNN':
      self.rnn = ManualNMRNN(self.I,self.H)
    self.decoder = nn.Linear(self.H, self.O)
    self.sparsity_lambda = sparsity_lambda

  def forward(self, inputs):
    hidden, _ = self.rnn(inputs)
    predictions = self.decoder(hidden)
    return predictions

  def compute_losses(self, predictions, targets):
    prediction_loss = nn.functional.cross_entropy(predictions, targets) #NB: this applies softmax itself
    #for sparsity we need to select the right weights to regularise
    sparsity_loss = 0 #init sparsity_loss
    for name, param in self.rnn.named_parameters():
      if 'bias' not in name:
        sparsity_loss += self.sparsity_lambda*torch.abs(param).sum()
    return prediction_loss, sparsity_loss




In [ ]:
from NM_TinyRNN.code.models import training
from importlib import reload; reload(training);
DATA_PATH = Path('./NM_TinyRNN/data/rnns')
model = TinyRNN(input_size=3, hidden_size=1, out_size=2, rnn_type='GRU', sparsity_lambda = SPARSITY_LAMBDA) # Example sizes
trainer = training.Trainer(DATA_PATH/'GRU')
trainer.fit(model,dataset)

In [ ]:
best_state_dict = torch.load(DATA_PATH/'GRU/1_unit_GRU_L1_0.0001_model_state.pth')
model.load_state_dict(best_state_dict)

In [ ]:
model.eval()
trial_by_trial_data = {}
n_batches, n_seq, n_inputs = dataset.inputs.shape
inputs_reshaped = dataset.inputs.reshape(n_batches*n_seq,n_inputs).unsqueeze(0)
with torch.no_grad():
    predictions = model(inputs_reshaped)
    hidden_state, gate_activations = model.rnn(inputs_reshaped, return_gate_activations = True)
# add hidden activations
for each_unit in range(model.H):
    trial_by_trial_data[f'hidden_{each_unit+1}'] = hidden_state[:,:,each_unit].flatten()
#add gate activations
for gate_label, activations in gate_activations.items():
    print(activations.shape)
    for each_unit in range(activations.shape[-1]):
        trial_by_trial_data[f'gate_{gate_label}_{each_unit+1}'] = activations[:,:,each_unit].flatten()
# add logit and probabilities data:
log_probs= predictions.log_softmax(dim=2)
logits = (log_probs[:,:,0] - log_probs[:,:,1]).flatten()
trial_by_trial_data['logit_value'] = logits
trial_by_trial_data['logit_change'] = torch.cat([torch.tensor([torch.nan]),(logits[1:]-logits[:-1])])
trial_by_trial_data['prob_A'] = torch.exp(log_probs[:,:,0]).flatten()
trial_by_trial_data['prob_b'] = torch.exp(log_probs[:,:,1]).flatten()

#add whether trial was used in training, validation, or evaluation
_,_,_, indices_dict = trainer._split_dataset(dataset)
for idx_label, idx_values in indices_dict.items():
    # Generate all trial indices for the given batches
    trial_indices = np.concatenate([np.arange(idx * n_seq, (idx + 1) * n_seq)
                                    for idx in idx_values])
    # Create boolean mask using isin
    all_trial_indices = np.arange(n_batches * n_seq)
    label_indices = np.isin(all_trial_indices, trial_indices)
    indices_dict[idx_label] = label_indices
trial_by_trial_data.update(indices_dict)
df = pd.DataFrame(trial_by_trial_data)
df

In [ ]:
hidden_state, gate_activations = model.rnn(inputs_reshaped, return_gate_activations = True)
gate_activations

In [ ]:
_, _, _, indices_dict = trainer._split_dataset(dataset)

for idx_label, idx_values in indices_dict.items():
    # Generate all trial indices for the given batches
    trial_indices = np.concatenate([
        np.arange(idx * n_seq, (idx + 1) * n_seq)
        for idx in idx_values])

    # Create boolean mask using isin
    all_trial_indices = np.arange(n_batches * n_seq)
    label_indices = np.isin(all_trial_indices, trial_indices)

    indices_dict[idx_label] = label_indices
indices_dict